In [20]:
%matplotlib inline

from numpythia import Pythia, STATUS, HAS_END_VERTEX, ABS_PDG_ID
from pyjet import cluster
import matplotlib.pyplot as plt
import numpy as np
import ROOT
from ROOT import gRandom, TH1, TH1D, cout, TCanvas
ROOT.gStyle.SetOptStat('')


In [21]:
selection = ((STATUS == 1) & ~HAS_END_VERTEX &
 (ABS_PDG_ID != 12) & (ABS_PDG_ID != 14) & (ABS_PDG_ID != 16))  #removing neutrinos

In [27]:
params = {
 "HardQCD:all": "on",
 "Beams:eCM": 100,
 "Beams:idA": 11,
 "Beams:idB": 2212,
 "PDF:lepton2gamma": "on",
 "Photon:Q2max":1.0, 
 "Photon:Wmin":10.0,
 "PhaseSpace:pTHatMin":5.0,
 "PhotonParton:all": "on",
 "Photon:ProcessType":0,
 "Main:timesAllowErrors":100,
}


In [23]:
#// Set the type of gamma-gamma process:
#    // 0 = mix of all below,
#    // 1 = resolved-resolved,
#    // 2 = resolved-direct,
#    // 3 = direct-resolved,
#    // 4 = direct-direct.
#np.dtype([('E', 'f8'), ('px', 'f8'), ('py', 'f8'), ('pz', 'f8'), ('mass', 'f8'),
#          ('prodx', 'f8'), ('prody', 'f8'), ('prodz', 'f8'), ('prodt', 'f8'),
#          ('pdgid', 'i4'), ('status', 'i4')])

In [28]:
pythia = Pythia(params=params)


 *-------  PYTHIA Process Initialization  --------------------------*
 |                                                                  |
 | We collide e- with p+ at a CM energy of 1.000e+02 GeV            |
 |                                                                  |
 |------------------------------------------------------------------|
 |                                                    |             |
 | Subprocess                                    Code |   Estimated |
 |                                                    |    max (mb) |
 |                                                    |             |
 |------------------------------------------------------------------|
 |                                                    |             |
 | g g -> g g                                     111 |   2.266e-05 |
 | g g -> q qbar (uds)                            112 |   5.674e-07 |
 | q g -> q g                                     113 |   7.486e-05 |
 | q q(bar)' -> q q

In [25]:
hpt = ROOT.TH1F("hpt","hpt",20,0,20.0)
heta = ROOT.TH1F("heta","heta",100,-5.0,5.0)
h_nch = ROOT.TH1F("h_nch","h_nch",200,0,200)
hpt.Sumw2()
heta.Sumw2()
h_nch.Sumw2()

Warning in <TROOT::Append>: Replacing existing TH1: hpt (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: heta (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_nch (Potential memory leak).


In [31]:
for event in pythia(events=1000):
    #if not event: continue
    
    #jets = cluster(event.all(selection), R=1, p=-1, ep=True).inclusive_jets(ptmin=5)
    #print('This events has # ', len(jets), 'jets')
    
    #for j in jets:
        #if j.pt>4.0:
        #    print(j.pt)
        #print ' jet' , j.pt, ' ' , j.eta,  ' ' , j.phi
    
    nch = 0
    for p in event.all(selection & (ABS_PDG_ID == 211)):
        pt = np.sqrt(p['px']*p['px']* + p['py']*p['py'])
        p3 = np.sqrt(p['px']*p['px']* + p['py']*p['py'] + p['pz']*p['pz'])
        eta = 0.5*np.log( (p3 + p['pz'])/(p3-p['pz']))
        if(pt<0.500): continue
        #print p['E'], ' ', p['pdgid'], ' ', pt
        #hpt.Fill(pt) 
        heta.Fill(eta)
        nch +=1 
    h_nch.Fill(nch)
#print ' '
    

RuntimeError: PYTHIA event generation aborted prematurely

In [ ]:
c = ROOT.TCanvas('c','c',1600,600)
h_nch.Draw()
c.Draw()
c.SaveAs("hpt.pdf")
